In [1]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import numpy as np
from altair import datum
import altair as alt
import shapely
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Import

Read in raw capacity data and locations from HHS

In [2]:
src = pd.read_csv('/Users/mhustiles/data/github/coronavirus-tracker/_notebooks/data/processed/hhs-icu-timeseries.csv', \
                  dtype={'ccn':str, 'total_covid_icu_patients':float})

In [3]:
sbcounty = src[src.county == 'San Bernardino']

In [4]:
sb_latest = sbcounty[sbcounty.week == sbcounty.week.max()].sort_values('pct_occupied_adult_icu_beds', ascending=False)

In [5]:
sbcounty[sbcounty.week == sbcounty.week.max()].sort_values('pct_occupied_adult_icu_beds', ascending=False)

,hospital,ccn,week,county,fips,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds
2125,VICTOR VALLEY GLOBAL MEDICAL CENTER,050517,2020-12-11,San Bernardino,71,10.857143,-2.285714,1.228571
2182,BARSTOW COMMUNITY HOSPITAL,050298,2020-12-11,San Bernardino,71,11.428571,0.000000,1.000000
2242,CHINO VALLEY MEDICAL CENTER,050586,2020-12-11,San Bernardino,71,9.142857,0.000000,1.000000
2750,MONTCLAIR HOSPITAL MEDICAL CENTER,050758,2020-12-11,San Bernardino,71,13.000000,0.000000,1.000000
3670,DESERT VALLEY HOSPITAL,050709,2020-12-11,San Bernardino,71,17.571429,0.000000,1.000000
3797,ST MARY MEDICAL CENTER,050300,2020-12-11,San Bernardino,71,18.142857,0.000000,1.000000
5310,SAN ANTONIO REGIONAL HOSPITAL,050099,2020-12-11,San Bernardino,71,27.142857,0.000000,1.000000
3675,REDLANDS COMMUNITY HOSPITAL,050272,2020-12-11,San Bernardino,71,9.857143,0.142857,0.992958
1372,KINDRED HOSPITAL RANCHO,052049,2020-12-11,San Bernardino,71,0.000000,0.428571,0.946429
5902,KAISER FOUNDATION HOSPITAL FONTANA,050140,2020-12-11,San Bernardino,71,54.285714,5.000000,0.941176


In [6]:
antelope_valley = src[src['hospital'] == 'ARROWHEAD REGIONAL MEDICAL CENTER']

In [7]:
antelope_valley.sort_values('week', ascending=False).to_csv('output/arrowhead.csv', index=False)

In [8]:
antelope_valley.pct_occupied_adult_icu_beds.mean()

0.8430332075545802

In [9]:
alt.Chart(antelope_valley).mark_line().encode(
    x=alt.X('week:T', axis=alt.Axis(format='', title='week', tickCount=3)),
    y = alt.Y('pct_occupied_adult_icu_beds', title=''),
).properties(width=400, height=400)

alt.Chart(...)

In [10]:
alt.Chart(sbcounty).mark_area(color='red').encode(
    x=alt.X('week:T', axis=alt.Axis(format='', title='week', tickCount=3)),
    y = alt.Y('pct_occupied_adult_icu_beds', title='', axis=alt.Axis(format='%', tickCount=5),scale=alt.Scale(domain=(0, 1))),
    facet=alt.Facet('hospital', columns=4),
).properties(width=100, height=100, title='% occupied ICU beds, by San Bernardino County hospital')

alt.Chart(...)

In [11]:
sbcounty_grouped = sbcounty.groupby(['county', 'week']).agg({'total_covid_icu_patients':'sum'}).reset_index()

In [12]:
alt.Chart(sbcounty_grouped).mark_line().encode(
    x=alt.X('week:T', axis=alt.Axis(format='', title='week', tickCount=3)),
    y = alt.Y('total_covid_icu_patients', title=''),
).properties(width=400, height=400)

alt.Chart(...)

In [13]:
sbcounty_grouped

,county,week,total_covid_icu_patients
0,San Bernardino,2020-07-31,182.047619
1,San Bernardino,2020-08-07,164.333333
2,San Bernardino,2020-08-14,140.238095
3,San Bernardino,2020-08-21,113.747619
4,San Bernardino,2020-08-28,102.976190
5,San Bernardino,2020-09-04,88.559524
6,San Bernardino,2020-09-11,66.452381
7,San Bernardino,2020-09-18,57.619048
8,San Bernardino,2020-09-25,52.285714
9,San Bernardino,2020-10-02,51.228571


---

In [14]:
counties = pd.read_csv('/Users/mhustiles/data/github/coronavirus-tracker/_notebooks/data/processed/benchmark-cases-timeseries.csv')

In [15]:
lately = pd.DataFrame(counties[counties['date'] > '2020-03-01'])

In [16]:
lately

,date,county,fips,new_confirmed_cases,new_confirmed_cases_seven_day_total,new_deaths,new_deaths_seven_day_total,new_confirmed_cases_seven_day_average,new_deaths_seven_day_average,population,new_confirmed_cases_per_100k,new_deaths_per_100k,new_confirmed_cases_seven_day_per_100k,new_deaths_seven_day_per_100k,new_confirmed_cases_seven_day_average_per_100k,new_deaths_seven_day_average_per_100k,new_confirmed_cases_seven_day_average_per_100k_percent_change,new_deaths_seven_day_average_per_100k_percent_change
36,2020-03-02,Alameda,1,0.0,1.0,0.0,0.0,0.142857,0.000000,1643700,0.000000,0.000000,0.060838,0.000000,0.008691,0.000000,-1.000000,NaN
37,2020-03-03,Alameda,1,1.0,2.0,0.0,0.0,0.285714,0.000000,1643700,0.060838,0.000000,0.121677,0.000000,0.017382,0.000000,-1.000000,NaN
38,2020-03-04,Alameda,1,0.0,2.0,0.0,0.0,0.285714,0.000000,1643700,0.000000,0.000000,0.121677,0.000000,0.017382,0.000000,-1.000000,NaN
39,2020-03-05,Alameda,1,0.0,2.0,0.0,0.0,0.285714,0.000000,1643700,0.000000,0.000000,0.121677,0.000000,0.017382,0.000000,-1.000000,NaN
40,2020-03-06,Alameda,1,1.0,3.0,0.0,0.0,0.428571,0.000000,1643700,0.060838,0.000000,0.182515,0.000000,0.026074,0.000000,-1.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19251,2020-12-18,Yuba,115,48.0,440.0,1.0,5.0,62.857143,0.714286,75493,63.582054,1.324626,582.835495,6.623131,83.262214,0.946162,0.042654,-1.0
19252,2020-12-19,Yuba,115,0.0,440.0,0.0,5.0,62.857143,0.714286,75493,0.000000,0.000000,582.835495,6.623131,83.262214,0.946162,0.198910,-1.0
19253,2020-12-20,Yuba,115,0.0,440.0,0.0,5.0,62.857143,0.714286,75493,0.000000,0.000000,582.835495,6.623131,83.262214,0.946162,0.198910,-1.0
19254,2020-12-21,Yuba,115,162.0,432.0,0.0,2.0,61.714286,0.285714,75493,214.589432,0.000000,572.238486,2.649252,81.748355,0.378465,0.043478,-1.0


In [17]:
lately.groupby(['county', 'date']).agg({'new_confirmed_cases_seven_day_per_100k':'sum'}).reset_index()

,county,date,new_confirmed_cases_seven_day_per_100k
0,Alameda,2020-03-02,0.060838
1,Alameda,2020-03-03,0.121677
2,Alameda,2020-03-04,0.121677
3,Alameda,2020-03-05,0.121677
4,Alameda,2020-03-06,0.182515
...,...,...,...
17163,Yuba,2020-12-18,582.835495
17164,Yuba,2020-12-19,582.835495
17165,Yuba,2020-12-20,582.835495
17166,Yuba,2020-12-21,572.238486


In [18]:
slim = lately[['date', 'county', 'new_confirmed_cases_seven_day_per_100k']]

In [19]:
slim_pivot = slim.pivot(index='date', columns='county', values='new_confirmed_cases_seven_day_per_100k').reset_index()

In [20]:
slim_pivot_counties = ['Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa',
       'Contra Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn', 'Humboldt',
       'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 'Lassen', 'Los Angeles',
       'Madera', 'Marin', 'Mariposa', 'Mendocino', 'Merced', 'Modoc', 'Mono',
       'Monterey', 'Napa', 'Nevada', 'Orange', 'Placer', 'Plumas', 'Riverside',
       'Sacramento', 'San Benito', 'San Bernardino', 'San Diego',
       'San Francisco', 'San Joaquin', 'San Luis Obispo', 'San Mateo',
       'Santa Barbara', 'Santa Clara', 'Santa Cruz', 'Shasta', 'Sierra',
       'Siskiyou', 'Solano', 'Sonoma', 'Stanislaus', 'Sutter', 'Tehama',
       'Trinity', 'Tulare', 'Tuolumne', 'Ventura', 'Yolo', 'Yuba']

In [21]:
slim_pivot['winner'] = slim_pivot[slim_pivot_counties].idxmax(axis=1)

In [25]:
slim_pivot[['date', 'winner',]].tail(10)

county,date,winner
286,2020-12-13,Lassen
287,2020-12-14,Lassen
288,2020-12-15,Lassen
289,2020-12-16,Lassen
290,2020-12-17,Lassen
291,2020-12-18,Lassen
292,2020-12-19,Lassen
293,2020-12-20,Lassen
294,2020-12-21,Lassen
295,2020-12-22,San Bernardino


In [23]:
slim_pivot['winner'].value_counts()

Imperial          69
Lassen            46
Kings             45
Santa Clara       23
Mono              21
Kern              19
Shasta            12
Los Angeles       11
Alpine             8
Merced             8
Santa Barbara      7
San Francisco      5
Riverside          5
Tehama             4
Fresno             3
Colusa             2
San Bernardino     1
San Joaquin        1
Tulare             1
Solano             1
San Benito         1
Glenn              1
Stanislaus         1
Calaveras          1
Name: winner, dtype: int64